In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [7]:
#dealing with mnist, 10 output channels requireed minm
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #28x28 (initially)
        #target rf ~ 28

        self.conv1 = nn.Sequential(  #26, rf = 3
            nn.Conv2d(1, 16, 3, padding=0, stride = 1, bias = False), #input -?(28) OUtput? = (28) RF = (3)
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.15)
        )


        self.conv3 = nn.Sequential( #22, #rf = 7
            nn.Conv2d(16, 32, 3, padding=0, stride = 1, bias = False),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.15)
        )

        #TB:  3 > 5> 7 > 9 > 10 > 12 > 12 > 28
        self.tb = nn.Sequential(     #we don't add anythign with tb
            nn.Conv2d(32, 10, 1, padding = 0, stride = 1, bias = False) #size = 11 , rf = 8
        )
        self.pool1 = nn.MaxPool2d(2,2)

        self.conv4 = nn.Sequential( #9 , #rf = 12
            nn.Conv2d(10, 10,3, padding=0, stride = 1, bias = False),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(0.15)
        )

        self.conv5 = nn.Sequential( #9, #rf = 12
            nn.Conv2d(10, 32, 3, padding=0, stride = 1, bias = False),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.15)
        )

        self.conv6 = nn.Sequential( #7, #rf = 16
            nn.Conv2d(32, 32 ,3, padding=0, stride = 1, bias = False),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.15)
        )

        self.conv7 = nn.Sequential( #8, #rf = 20
            nn.Conv2d(32, 16 ,1, padding=0, stride = 1, bias = False),

        )

        self.bn = nn.AdaptiveAvgPool2d(1) #uses kernel of size 8, s = 2 => jout =4, rout = 18 + 14

        self.conv8 = nn.Conv2d(16, 10, 1, padding = 0)

    def forward(self, x):

        x = self.conv1(x)
        x = self.conv3(x)
        x = self.tb(x)
        x = self.pool1(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.bn(x)
        x = self.conv8(x)
        x = x.view(x.size(0), -1)
        return F.log_softmax(x)

#CB2
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 32, 24, 24]           4,608
              ReLU-6           [-1, 32, 24, 24]               0
       BatchNorm2d-7           [-1, 32, 24, 24]              64
           Dropout-8           [-1, 32, 24, 24]               0
            Conv2d-9           [-1, 10, 24, 24]             320
        MaxPool2d-10           [-1, 10, 12, 12]               0
           Conv2d-11           [-1, 10, 10, 10]             900
             ReLU-12           [-1, 10, 10, 10]               0
      BatchNorm2d-13           [-1, 10, 10, 10]              20
          Dropout-14           [-1, 10,

<ipython-input-7-265096a801d1>:82: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [8]:
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
torch.manual_seed(1)
batch_size = 128


kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-7.0, 7.0), fill=(1,)),  #adding iamge augmentation
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        y_pred = model(data)
        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    Accuracy = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return Accuracy

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience = 10)

for epoch in range(1, 20):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test_accuracy = test(model, device, test_loader)
    scheduler.step(test_accuracy)

EPOCH: 1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-7-265096a801d1>:82: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
Loss=0.08764037489891052 Batch_id=468 Accuracy=83.17: 100%|██████████| 469/469 [00:25<00:00, 18.66it/s]



Test set: Average loss: 0.0737, Accuracy: 9794/10000 (98%)

EPOCH: 2


Loss=0.05998275801539421 Batch_id=468 Accuracy=97.23: 100%|██████████| 469/469 [00:26<00:00, 17.79it/s]



Test set: Average loss: 0.0408, Accuracy: 9880/10000 (99%)

EPOCH: 3


...


Test set: Average loss: 0.0225, Accuracy: 9936/10000 (99%)

EPOCH: 18


Loss=0.0073632062412798405 Batch_id=468 Accuracy=99.01: 100%|██████████| 469/469 [00:25<00:00, 18.30it/s]



Test set: Average loss: 0.0184, Accuracy: 9937/10000 (99%)

EPOCH: 19


Loss=0.022108137607574463 Batch_id=468 Accuracy=99.06: 100%|██████████| 469/469 [00:26<00:00, 17.90it/s]



Test set: Average loss: 0.0186, Accuracy: 9943/10000 (99%)

